# KOPIS 공연추천 시스템 테스트

KOPIS OpenAPI를 활용하여 공연 정보를 수집하고, OCR과 FastText를 이용한 추천 시스템을 구현합니다.

## 필요한 라이브러리 설치

## 라이브러리 임포트

In [13]:
pip install sentence-transformers kss

     ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
     ---------------------------------------- 1.1/1.1 MB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached scikit_learn-1.6.0-cp312-cp312-win_amd64.whl.metadata (15 kB)
     ---------------------------------------- 0.0/26.4 MB ? eta -:--:--
     --- ------------------------------------ 2.1/26.4 MB 9.8 MB/s eta 0:00:03
     ----- ---------------------------------- 3.9/26.4 MB 9.4 MB/s eta 0:00:03
     -------- ------------------------------- 5.8/26.4 MB 9.5 MB/s eta 0:00:03
     ----------- ---------------------------- 7.6/26.4 MB 9.2 MB/s eta 0:00:03
     ------------- -------------------------- 9.2/26.4 MB 9.2 MB/s eta 0:00:02
     ----------------- ---------------------- 11.3/26.4 MB 9.3 MB/s eta 0:00:02
     -------------------- ------------------- 13.6/26.4 MB 9.5 MB/s eta 0:00:02
     ----------------------- ----------------

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [54 lines of output]
      running egg_info
      writing lib\PyYAML.egg-info\PKG-INFO
      writing dependency_links to lib\PyYAML.egg-info\dependency_links.txt
      writing top-level names to lib\PyYAML.egg-info\top_level.txt
      Traceback (most recent call last):
        File "c:\Users\USER\anaconda3\envs\ds_env\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "c:\Users\USER\anaconda3\envs\ds_env\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\USER\anaconda3\envs\ds_env\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_whee

In [17]:
import cv2
import requests
import pandas as pd
import numpy as np
from PIL import Image, ImageEnhance
import pytesseract
from io import BytesIO
from gensim.models import FastText as FastText
import re
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET
from typing import Optional, Dict, Any, List
from dotenv import load_dotenv
import os
from sentence_transformers import SentenceTransformer

c:\Users\USER\anaconda3\envs\ds_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## KOPIS API 클라이언트 클래스 정의

In [18]:
# .env 파일 로드
load_dotenv()

# API 키를 .env 파일에서 가져오기
KOPIS_API_KEY = os.getenv('KOPIS_API_KEY')

In [19]:
class KopisAPI:
    def __init__(self, service_key):
        self.service_key = service_key
        self.base_url = "http://www.kopis.or.kr/openApi/restful"
    
    def get_performance_list(self, start_date, end_date):
        """공연 목록 조회"""
        url = f"{self.base_url}/pblprfr"
        params = {
            'service': self.service_key,
            'stdate': start_date,
            'eddate': end_date,
            'rows': 100,
            'cpage': 1
        }
        response = requests.get(url, params=params)
        root = ET.fromstring(response.content)
        
        performances = []
        for db in root.findall('.//db'):
            perf = {}
            for child in db:
                perf[child.tag] = child.text
            performances.append(perf)
        
        return performances
    
    def get_performance_detail(self, mt20id: str) -> Optional[Dict[str, Any]]:
        """공연 상세정보 조회 - 포스터와 소개이미지 모두 처리"""
        url = f"{self.base_url}/pblprfr/{mt20id}"
        params = {'service': self.service_key}
        
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            
            root = ET.fromstring(response.content)
            db = root.find('.//db')
            
            if db is None:
                return None
                
            detail = {}
            for elem in db:
                if elem.tag == 'styurls':
                    # XML 구조 디버깅
                    print(f"styurls element found for {mt20id}")
                    print(f"styurls content: {ET.tostring(elem, encoding='unicode')}")
                    
                    # 소개이미지 목록 추출 (수정된 XPath)
                    urls = []
                    for styurl in elem.findall('styurl'):
                        if styurl.text and styurl.text.strip():
                            print(f"Found image URL: {styurl.text}")
                            urls.append(styurl.text.strip())
                    detail['styurls'] = urls
                else:
                    if elem.text and elem.text.strip():
                        detail[elem.tag] = elem.text.strip()
                    
            # 디버깅을 위한 출력
            if 'styurls' in detail:
                print(f"Total styurls found for {mt20id}: {len(detail['styurls'])}")
            else:
                print(f"No styurls found for {mt20id}")
                
            return detail
            
        except Exception as e:
            print(f"API 요청 오류: {e}")
            return None

## 텍스트 처리 클래스 정의

In [20]:
class TextProcessor:
    def __init__(self):
        """
        텍스트 프로세서 초기화
        - OCR 설정 유지
        - SentenceTransformer 모델 추가
        - 임베딩 캐시 추가
        """
        pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
        
        # SentenceTransformer 모델 초기화 (경량 다국어 모델)
        try:
            self.st_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
            self.use_transformers = True
            self.embedding_dim = 384  # MiniLM 모델의 출력 차원
        except ImportError:
            print("SentenceTransformers를 사용할 수 없습니다. FastText로 대체됩니다.")
            self.use_transformers = False
            self.model = None
            self.embedding_dim = 100  # FastText 기본 차원
            
        # 임베딩 캐시 초기화
        self.embedding_cache = {}
        
        # 배치 처리 설정
        self.batch_size = 32
    
    def enhance_image(self, img):
        """이미지 품질 개선 - 기존 코드 유지"""
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(2.0)
        enhancer = ImageEnhance.Sharpness(img)
        img = enhancer.enhance(2.0)
        return img
    
    def preprocess_image(self, img_array):
        """OpenCV 이미지 전처리 - 기존 코드 유지"""
        gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
        denoised = cv2.fastNlMeansDenoising(gray)
        _, binary = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        processed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
        return processed
    
    def extract_valid_sentences(self, text: str) -> List[str]:
        """유효한 문장 추출"""
        try:
            # 문장 분리
            sentences = kss.split_sentences(text)
            
            # 유효한 문장 필터링
            valid_sentences = []
            for sent in sentences:
                # 최소 길이 체크
                if len(sent) < 10:
                    continue
                    
                # 한글/영문 비율 체크
                char_ratio = sum(1 for c in sent if c.isalnum()) / len(sent)
                if char_ratio < 0.5:
                    continue
                    
                valid_sentences.append(sent)
                
            return valid_sentences
        except:
            return [text] if text.strip() else []

    def extract_text_from_image(self, image_url):
        """이미지에서 텍스트 추출 - 기존 코드 유지 및 문장 필터링 추가"""
        try:
            print(f"이미지 다운로드 시도: {image_url}")
            response = requests.get(image_url)
            img = Image.open(BytesIO(response.content))
            
            # 이미지 전처리 (기존 코드)
            if img.format == 'GIF':
                img = img.convert('RGB')
                
            target_width = 1000
            width_percent = (target_width / float(img.size[0]))
            target_height = int(float(img.size[1]) * float(width_percent))
            img = img.resize((target_width, target_height), Image.Resampling.LANCZOS)
            
            enhanced_img = self.enhance_image(img)
            img_array = np.array(enhanced_img)
            processed_img = self.preprocess_image(img_array)
            
            # OCR 수행
            extracted_text = pytesseract.image_to_string(
                processed_img,
                lang='kor+eng',
                config='--oem 3 --psm 6'
            )
            
            # 유효한 문장 필터링
            valid_sentences = self.extract_valid_sentences(extracted_text)
            cleaned_text = ' '.join(valid_sentences)
            
            print(f"추출된 총 텍스트 길이: {len(cleaned_text)}")
            print(f"텍스트 샘플: {cleaned_text[:200]}...")
            
            return cleaned_text
            
        except Exception as e:
            print(f"이미지 처리 중 오류 발생: {str(e)}")
            return ""

    def get_text_vector(self, text: str) -> np.ndarray:
        """텍스트 벡터화 - SentenceTransformers 사용"""
        if not text.strip():
            return np.zeros(self.embedding_dim)
            
        # 캐시된 임베딩이 있는지 확인
        cache_key = hash(text)
        if cache_key in self.embedding_cache:
            return self.embedding_cache[cache_key]
            
        if self.use_transformers:
            # 문장 분리 및 배치 처리
            sentences = self.extract_valid_sentences(text)
            if not sentences:
                return np.zeros(self.embedding_dim)
                
            # 배치 단위로 임베딩 생성
            embeddings = []
            for i in range(0, len(sentences), self.batch_size):
                batch = sentences[i:i + self.batch_size]
                batch_embeddings = self.st_model.encode(batch, convert_to_numpy=True)
                embeddings.extend(batch_embeddings)
                
            # 평균 임베딩 계산
            embedding = np.mean(embeddings, axis=0)
            
        else:
            # FastText fallback (기존 코드)
            if self.model is None:
                return np.zeros(self.embedding_dim)
                
            words = text.split()
            word_vectors = [self.model.wv[word] for word in words if word in self.model.wv]
            if not word_vectors:
                return np.zeros(self.embedding_dim)
            embedding = np.mean(word_vectors, axis=0)
            
        # 임베딩 캐시에 저장
        self.embedding_cache[cache_key] = embedding
        return embedding

## 공연 추천 시스템 클래스 정의

In [21]:
class PerformanceRecommender:
    def __init__(self, api_client, text_processor):
        self.api_client = api_client
        self.text_processor = text_processor
        self.performances_df = None
        self.batch_size = 16
        
    def collect_performance_data(self, days=30):
        """공연 데이터 수집 및 처리"""
        # 기존 데이터 수집 로직
        performances = []
        perf_list = self.api_client.get_performance_list(
            datetime.now().strftime("%Y%m%d"),
            (datetime.now() + timedelta(days=days)).strftime("%Y%m%d")
        )
        
        # 배치 단위로 데이터 처리
        for i in range(0, len(perf_list), self.batch_size):
            batch = perf_list[i:i + self.batch_size]
            for perf in batch:
                mt20id = perf['mt20id']
                detail = self.api_client.get_performance_detail(mt20id)
                
                if detail:
                    # 텍스트 추출 및 처리
                    texts = []
                    
                    # 포스터 텍스트
                    if 'poster' in detail and detail['poster']:
                        poster_text = self.text_processor.extract_text_from_image(detail['poster'])
                        texts.append(poster_text)
                    
                    # 소개이미지 텍스트
                    if 'styurls' in detail and isinstance(detail['styurls'], list):
                        for img_url in detail['styurls']:
                            if img_url and img_url.startswith('http'):
                                intro_text = self.text_processor.extract_text_from_image(img_url)
                                if intro_text:
                                    texts.append(intro_text)
                    
                    all_text = ' '.join(texts)
                    valid_sentences = self.text_processor.extract_valid_sentences(all_text)
                    processed_text = ' '.join(valid_sentences)
                    
                    performances.append({
                        'mt20id': mt20id,
                        'title': detail.get('prfnm', ''),
                        'plot': processed_text
                    })
        
        self.performances_df = pd.DataFrame(performances)
        return self.performances_df
    
    def get_recommendations(self, user_plot, top_n=5):
        """사용자 입력 기반 공연 추천"""
        if self.performances_df is None:
            raise ValueError("공연 데이터를 먼저 수집하세요.")
            
        # 사용자 입력 벡터화
        user_vector = self.text_processor.get_text_vector(user_plot)
        
        # 배치 단위로 유사도 계산
        similarities = []
        for i in range(0, len(self.performances_df), self.batch_size):
            batch = self.performances_df.iloc[i:i + self.batch_size]
            batch_similarities = []
            for _, row in batch.iterrows():
                plot_vector = self.text_processor.get_text_vector(row['plot'])
                similarity = np.dot(user_vector, plot_vector) / (
                    np.linalg.norm(user_vector) * np.linalg.norm(plot_vector)
                )
                batch_similarities.append(similarity)
            similarities.extend(batch_similarities)
            
        self.performances_df['similarity'] = similarities
        recommendations = self.performances_df.nlargest(top_n, 'similarity')
        return recommendations[['title', 'similarity']]


## 시스템 테스트

아래 셀에서 실제 테스트를 수행합니다. API 키를 설정하고 실행해보세요.

In [22]:
# 테스트 코드
if __name__ == "__main__":
    # API 키 설정
    SERVICE_KEY = KOPIS_API_KEY
    
    # 시스템 초기화
    api_client = KopisAPI(SERVICE_KEY)
    text_processor = TextProcessor()
    recommender = PerformanceRecommender(api_client, text_processor)
    
    # 데이터 수집
    print("데이터 수집 중...")
    performances_df = recommender.collect_performance_data()
    print("\n수집된 공연 데이터:")
    print(performances_df.head())

c:\Users\USER\anaconda3\envs\ds_env\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


데이터 수집 중...
styurls element found for PF256564
styurls content: <styurls>
            <styurl>http://www.kopis.or.kr/upload/pfmIntroImage/PF_PF256564_250102_0144061.jpg</styurl>
            <styurl>http://www.kopis.or.kr/upload/pfmIntroImage/PF_PF256564_250102_0144060.jpg</styurl>
        </styurls>
        
Found image URL: http://www.kopis.or.kr/upload/pfmIntroImage/PF_PF256564_250102_0144061.jpg
Found image URL: http://www.kopis.or.kr/upload/pfmIntroImage/PF_PF256564_250102_0144060.jpg
Total styurls found for PF256564: 2
이미지 다운로드 시도: http://www.kopis.or.kr/upload/pfmPoster/PF_PF256564_250102_134406.jpg
추출된 총 텍스트 길이: 26
텍스트 샘플: KJror Hon
oa
「     PM 글호애
...
이미지 다운로드 시도: http://www.kopis.or.kr/upload/pfmIntroImage/PF_PF256564_250102_0144061.jpg
추출된 총 텍스트 길이: 26
텍스트 샘플: KJror Hon
oa
「     PM 글호애
...
이미지 다운로드 시도: http://www.kopis.or.kr/upload/pfmIntroImage/PF_PF256564_250102_0144060.jpg
추출된 총 텍스트 길이: 431
텍스트 샘플: 어레. ELF                                   그 후미
.                           

In [23]:
pd.set_option('display.max_colwidth', None)

In [24]:
performances_df

,mt20id,title,plot
0,PF256564,한우리 피아노 독주회,"KJror Hon\noa\n「 PM 글호애\n KJror Hon\noa\n「 PM 글호애\n 어레. ELF 그 후미\n. |\n기 「\nsot = 7 o¢g =\n. . ""te . ~\nPPOGRAM\nE.Griegas 나이 a Bas. upd\nL.v. Beethoven. - 1 - 피기 Feat Me SPM 1 다 14\nintermission\nJ.S.Bach:---* WaT red niet Bak |\nPpshedt ped Tse 지이 Da tame, PA 7\nTeilanes, Bs Dar Yohei be cra\nR. Schumann -42- Tava barndie ce\n애리\nwenn\n"
1,PF256563,"피아니스트 김경현의 클래식 힐링 콘서트 4, 열두달 음악으로 마음챙김","： 내 인 두 단\n> 월 '음악으로\n도 =. , o ~ ` we . ~.\n김경현의 | pf ay\ne # ~\n~ 대 때 . |\nfulness Musiée. EAN\n: wren BU 치\n> 복과 cores sarin eers sear ctevnem\n며~~ 알다 가\n @\n「 ~, aed\n* -> © o-o 2\nIe 22891 RN\n7; t eee\n - naraton JO 지\n* fi\n’ PS\n더 Cee 혹유1409626이고 미고 Pom, 욱일 os OO 29d BoM,\n： 주 1 * nmap wr HS a a\n|\n| … 000 : . .\n"
2,PF256561,코코아 한 잔을 마시면 우리는 영원히 행복해집니다,"다 yo, ue ‘ .\n\n: 2 20260123.91-이60000\noT Sa\natts | pe\n\n ~\n 다 yo, ue ‘ .\n\n: 2 20260123.91-이60000\noT Sa\natts | pe\n\n ~\n + 시눌지스 +\n+ Wei +\n. ME ;\n ~ 더\n- a “op Seto} 전더스는 신야야. Gat w? 안채.""\n+ 관람한4 +\n+ 접근성한 +\n+ 터시뚜어안내 +\n+ 84 +\nSTAGE\n4 2 내 4 호 $ 7 ‘ ' ‘0\n7 4 8146 B® 7 ew 8\nne - 22 98 고 32 32 개 7\n과식 ete a 음딩피리, 온라인 에지 옹 25서, 99 주 Odd 연식 열판 St\n6\n+ PtH +\n+ BSB +\n“ 24,000 8\nsee 주면 Wel (40%) W5000 원 / HEF Li 거주자 Wd 찍광인 포항\n다 36,000 월 / 만 70 미만\n162 오시오 오는 ]\nHE 할인 2039 8\n고기 배설린 (2099 veapoe at. mamasa mea\n광매연 Wel (40%)\nSe SO ee\n“Ute 지구를 2 지켜. 시키고 AAS 않고,”\n+ Sat +\nea\nc\n|\nI\n™ me.\n+ SRA +\nat\n\\ni?\n“** me\n+ WASISIY +\n= 에 am - .\n“= 4 2000 pon wae\nue a 2000 18 waa\n14100 we wes\n2000 tite ome\n“그램에도 MD 지키는 여유는 SURO 였어.""\n+ BEDI AH +\nme 」 ' a 이\n"
3,PF256560,테너 이승묵 독창회,"ee ~\nPy ~\n~\n~\nY\n ee ~\nPy ~\n~\n~\nY\n q\n때\n| 브다구 Bete\n“ ""fet wa\nuy aba ede wake\nTeds eed\nede ea,\n"
4,PF256559,백설공주 [서울 미아],"란미바가족!뮤지컬|\n2025년1월31일(금)>2원2일(일)\nCULE Cod Ee CPTI SILT, 1714\n연내뱅화세10[퍼10증 Fors\n재 | 34504 eS 이! | 0\n 라미브)가족!뮤지럴|\nㄷ\n 주 |\n2025년1월31일(금)>2월2일(일)\n1공연신2[금도일01브91041, 174\n연버뻐아페비마끔0[!층 문화을\n3a | ACURMY BAPASy 28 | EVN | Wena 06000\nEY =\nNp! NLA LN LN NG/\n/\/SYNOPSIS /\n0 Ny! ) 백섬공주의 새엄마인 OHS ' 시\n/\ 1 ¢§ 마순거물에게 세상메서 A \\n\ / A 누가 가잠 마름다문지 물었다 \ /\n내 마손거운은 백섬공주라고 Sct ) 시\n/ 나 하가난 마녀는 사남꾼른 시켜 ^ \\n ㅣ"" f 백섬곰주문 SOE AAC. 년 a!\nJ\ - 마지만 사냥꾼이 백섬공주를 삼려주어 : aN\noy 깊은 숲속으로 간 백설공주는 때\n/ » 그곳에서 난쟁이들을 만나는데 ~ \g/\n/ ㆍ Fp. 안 ise ames 변장마며 \ /\n\ iW SANTIS 101712 OFC... A\n$ iol 백섬공주는 1B SITIOS 양\n/ | U7 \/N\\nak y NOY . ~ . fn of\nVa DETAILS #1 06 「\n서머 [원작의 내용 과 교문적인 내용) ^\n\ / /\n& SS | &\n/ \ / 5 aZ \\n\ 7 _ \/\n@ - 건즈딘 스포리인 브선곤주는 : &\nZA) 이미드이기 판다지안 으소아 즌거운은 ^ \\n\ / 신/인것인니다 \ /\n에 그리고 익숙한 줄거리들 더묵 재미있고 Y 2\n/ : 7\nBy 살검나게 표연마여 「 .\n 「애 > xz BASS 줄것입니다 yo \g/\n7 \ 세 FA \ SN\n\ WA RN /\na ~ Ns 7 = 고 - , a\n20 DETAILS #2 CURR,\n{ 실감나는 영상과 무대 .\n 13곡의 뮤지컬 넘버\n\ $ SS J a!\n^ 노시 Na\n니니 ONY\n~ / Biel 제직안 배겸과 : / \\n000니순이 익지저럼 MHZIEION 그러시는것이미민뿔00\n< > 30로 SAO 배경처럼 OHSS 날고, 7 \,\n/ \ Pie L\\n\ / ar \ /\nSam? «ENS XIOHLIDA EILICE , &\n/ \ 작곡.편곡 믹심과 여러 배경음악음 7 \\n\ /@ 최고의 무지선든미 직접 제작아고 SCONE\n& : 머디서도 든을수 없는 음악을 선보였습!삐따, *\n/ \ 7 Seb 0H OWISS 신나고 712201 / \\n\ / NVouSele! 수 있슨니다: A /\ /\n¢. 9 <\n/ | / ^ /\n< 곰연사진\n\/\.7y TN\n- 에기 a, 3\n|\n\\n‘AS 2’\n7 ~ ~ ^ \ SN\nUs 붕북구 EAP 315\n초년 DIOMPIZIS GH S201 49510\nA 22 a a\n"
...,...,...,...
95,PF256341,"춘천시립합창단 모닝콘서트 PAUSE 1: 겨울, 그리고 다가올 봄","Vie Rhy tsrweime 20a suante rauses 0: y 를\nas AS, Ae\n ae at 1 SOW AN\n\nf. , 춘천문화예술회관 4 '\n$5: 타스. 0 2025.01.22.(수) 11:00 AM, Si, :\n43 )\n\n0 . ㆍ . I\n wey TT EISEN 3023 모닝콘서느 PAUSED 베버\n+ ae 2 그리고 | 사사\n< ¢ . 차 춘천문화예술회관 . A i\n| y~ 2025.01.22. (4) 11:00 AM. se, :\n"
96,PF256340,제10회 대구국제청소년오케스트라 정기연주회,"DADE AT ED 내 비비 마타 ~~\n2 0 27 5 대구국재정소너오케스트라\n=\n시 녀으아 외\nㅣ” 82 1\nAho “Eo 결 금교 14011: =\n주설바르|마 소극장\n. * te .* ‘\n별 , . r\n—\nog SIR Crs\nPOND BARR eek ia\n기 (태아 아이 RE 으으 빠 pe\n DADE AT ED 내 비비 마타 ~~\n2 0 27 5 대구국재정소너오케스트라\n=\n시 녀으아 외\nㅣ” 82 1\nAho “Eo 결 금교 14011: =\n주설바르|마 소극장\n. * te .* ‘\n별 , . r\n—\nog SIR Crs\nPOND BARR eek ia\n기 (태아 아이 RE 으으 빠 pe\n 025 UPA AMOE A= 그\nQW MR TIM SV Oe | ㆍ\n2 ey 0 2,\nSOG (1 10일 SaS ved) : os\nSSM 285\ni 

In [7]:
# 모델 학습
print("모델 학습 중...")
recommender.prepare_model()

모델 학습 중...


NameError: name 'recommender' is not defined

In [8]:
# 추천 테스트
test_input = "조선시대와 현대의 만남"
print(f"테스트 입력: {test_input}")

recommendations = recommender.get_recommendations(test_input)
print("\n추천 결과:")
display(recommendations)

테스트 입력: 조선시대와 현대의 만남


NameError: name 'recommender' is not defined

## 결과 분석

1. 추천된 공연들의 유사도 점수 분포
2. OCR 텍스트 추출 품질
3. FastText 모델의 성능